<a href="https://colab.research.google.com/github/muffin-head/netflixReccommendationEngine/blob/main/netflixReccommendationEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning':0})
import seaborn as sns
sns.set_style('whitegrid')
import os
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
:start= datetime.now()
if not os.path.isfile('data,csv'):
  data=open('data.csv',mode='w')
  row=list()
  files=['/content/drive/MyDrive/archive/combined_data_1.txt','/content/drive/MyDrive/archive/combined_data_2.txt','/content/drive/MyDrive/archive/combined_data_3.txt','/content/drive/MyDrive/archive/combined_data_4.txt']
  for file in files:
    with open(file) as f:
      for line in f:
        line=line.strip()
        if line.endswith(':'):
          movie_id=line.replace(':','')
        else:
          for i in line.split(','):
            row.append(i)
            row.insert(0,movie_id)
            data.write(','.join(row))
            data.write('\n')
    data.close()
datetime.now()-start

In [5]:
start = datetime.now()
if not os.path.isfile('data.csv'):
    # Create a file 'data.csv' before reading it
    # Read all the files in netflix and store them in one big file('data.csv')
    # We re reading from each of the four files and appendig each rating to a global file 'train.csv'
    data = open('data.csv', mode='w')

    row = list()
    files=['/content/drive/MyDrive/archive/combined_data_1.txt','/content/drive/MyDrive/archive/combined_data_2.txt','/content/drive/MyDrive/archive/combined_data_3.txt','/content/drive/MyDrive/archive/combined_data_4.txt']

    for file in files:
        print("Reading ratings from {}...".format(file))
        with open(file) as f:
            for line in f:
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
        print("Done.\n")
    data.close()
print('Time taken :', datetime.now() - start)

Reading ratings from /content/drive/MyDrive/archive/combined_data_1.txt...
Done.

Reading ratings from /content/drive/MyDrive/archive/combined_data_2.txt...
Done.

Reading ratings from /content/drive/MyDrive/archive/combined_data_3.txt...
Done.

Reading ratings from /content/drive/MyDrive/archive/combined_data_4.txt...
Done.

Time taken : 0:03:36.026040


In [4]:
dataframe=pd.read_csv('/content/drive/MyDrive/data.csv',sep=',',names=['movie','user','rating','date'])



In [ ]:
dataframe.date=pd.to_datetime(dataframe.date)
dataframe.sort_values(by='date', inplace=True)


In [ ]:
print(sum(dataframe.isnull().any()))
print(dataframe.duplicated(['movie','user','rating','date']))
np.unique(dataframe['movie'])
np.unique(dataframe['user'])
np.unique(dataframe['date'])
print(dataframe.shape[0])

In [ ]:

if not os.path.isfile('train.csv'):
  dataframe.iloc[:int(dataframe.shape[0]*0.80)].to_csv('train.csv',index=False)
if not os.path.isfile('test.csv'):
  dataframe.iloc[int(dataframe.shape[0]* 0.80):].to_csv('test.csv',index=False)

In [2]:
train_df=pd.read_csv('train.csv',parse_dates=['date'])
test_df=pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [9]:
train_df.head()

,movie,user,rating,date
0,10341,510180,4,1999-11-11
1,1798,510180,5,1999-11-11
2,10774,510180,3,1999-11-11
3,8651,510180,2,1999-11-11
4,14660,510180,2,1999-11-11


In [10]:
train_df.shape

(80384405, 4)

# Find out how often rating are given by a user

In [ ]:
# method to make y-axis more readable
def human(num, units = 'M'):
    units = units.lower()
    num = float(num)
    if units == 'k':
        return str(num/10**3) + " K"
    elif units == 'm':
        return str(num/10**6) + " M"
    elif units == 'b':
        return str(num/10**9) +  " B"

In [ ]:
fig, ax = plt.subplots()
plt.title('Distribution of ratings over Training dataset', fontsize=15)
sns.countplot(train_df.rating)
ax.set_yticklabels([human(item, 'M') for item in ax.get_yticks()])
ax.set_ylabel('No. of Ratings(Millions)')

plt.show()

<IPython.core.display.Javascript object>